In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tvb.simulator.lab as tsl
import mne_connectivity 
import tvb
from scipy import signal
from importlib import reload
import pandas as pd
from sklearn.decomposition import PCA
import seaborn as sns

from sklearn.cluster import KMeans
import scipy 
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from scipy.cluster.hierarchy import dendrogram, linkage
import pandas as pd
import netplotbrain

import functions as fn
reload(fn)

In [ ]:
def DFC_visu_pipeline(data, cor_method, num_clusters=3, window_size=2000, overlap=1000,tmax=30, intrahemispheric=False, square=[0,0],animated_visu=True,ts=0.5):
    '''
    This function performs visualisations regarding Dynamical Functional connectivity (DFC). 

    ------------
    INPUTS
    ------------

    data : array_like, shape = (signal_duration/timestep,num_regions) 
        The signal data, typically eeg signal or source level signal. 
    cor_method : string
        The correlation method to build the connectivity matrices. Can be chosen over 'Pearson', 'Spearman', 'Coherence', 'Phase-lock' and 'wpli'.
    num_clusters : int, default = 3
        Number of clusters for the k-means clustering.
    window_size : int, default = 2000  
        window_duration[ms]/timestep[ms]
    overlap : int or float, default = 1000 
        Time of overlap [ms] between two succint windows. 
    tmax : int, default = 30
        Time of the simulation, for visualisation purpose. 
    intrahemispheric : bool, default = False
        Visualisation of clustering of a single square of the connectivity matrix. 
    square : 2x2 list, default = [0,0]
        If intrahemispheric = True, specifies the square of the matrix to perform the analysis on. 
    animated_visu : bool, default = True
        Plot and save an animated plot showing the evolution of correlation matrices and pca visualisation. 


    ------------
    OUTPUTS
    ------------
    closest_mats : array_like, shape = (num_clusters, num_regions,num_regions)
        Closest connectivity matrix to every cluster. 
    most_repeating_states : array_like, shape = (num_clusters, num_regions,num_regions)
         Kmeans clusters centroids. 
    correlation_matrices : array_like, shape = (num_windows, num_regions,num_regions)
        Set of correlation matrics, one per window. 
    cluster_labels : list, len = num_clusters
        Cluster labels.
    distances : array_like, shape = (num_clusters, num_windows)
         Matrix storing the distances between each connectivity matrix and each cluster. 
    pca : Sklearn PCA object.
        PCA object used for the connectivity matrices dimension reduction.   
    reduced_data : array_like
        3D PCA reduced data.
    '''
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,num_clusters = fn.build_FCmat(data,cor_method,num_clusters,window_size,overlap,tmax,intrahemispheric,square=square)
    distances = fn.compute_clst_dist(most_repeating_states, correlation_matrices, cluster_labels,tmax,num_clusters) 
    pca, reduced_data = fn.plot_PCA(correlation_matrices, most_repeating_states, cluster_labels,num_clusters)
    if animated_visu : 
        fn.animated_plot(correlation_matrices,reduced_data,cluster_labels)

    return closest_mats,most_repeating_states, correlation_matrices, cluster_labels, distances, pca, reduced_data
    


# Define paths

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'

# Effects of window len

In [ ]:
Wins=[0.5,1,1.5,2,3,4,5,8,10] #s
timestep = 0.5 #ms
Wins=[i/timestep for i in Wins]
# Number of clusters (most repeating states)
num_clusters = 3
mats=np.zeros((len(Wins), num_clusters,76,76))
for i,w in enumerate(Wins):
    data = np.load(f'{path_save}/test1.5minsim_dt_0.5_G_{7}_sigma_1e-07.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 80000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    methods=['Pearson' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels, distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth,window_size=w*1000,overlap=w*500,tmax=t2/1000,animated_visu=False)
        mats[i]=closest_mats
    

# 40 Minutes stim

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/20minsim_dt_0.5_G_{7}_sigma_1e-07.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 1200000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth, window_size=10000,overlap=5000,tmax=t2/1000)

## Flipping hemispheres

In [ ]:
# 3D PCA
num_clusters=3
np.random.seed=42
rmd_idx=np.random.randint(1,957,10)
ext_data=np.concatenate((correlation_matrices,most_repeating_states))
data=np.array(ext_data).reshape(len(correlation_matrices)+num_clusters,5776)
#flipped_mtc=[inter_flip(mtx) for mtx in np.array(correlation_matrices)[rmd_idx]]
flipped_mtc=[fn.intra_flip(mtx) for mtx in np.array(correlation_matrices)[rmd_idx]]
pca_ref=PCA(n_components=3)
reduced_data = pca_ref.fit_transform(data)
flip_rshp = np.array(flipped_mtc).reshape(len(flipped_mtc),5776)
flip_pca=pca_ref.transform(flip_rshp)
centroids = reduced_data[-num_clusters:]

fig = go.Figure(data=[go.Scatter3d(x=reduced_data[:-3, 0], y=reduced_data[:-3, 1], z=reduced_data[:-3, 2],
                                mode='markers', marker=dict(color='black'), showlegend=False)])

fig.add_trace(go.Scatter3d(x=reduced_data[rmd_idx, 0], y=reduced_data[rmd_idx, 1], z=reduced_data[rmd_idx, 2],
                                mode='markers', marker=dict(color='red'), name='reference points'))

fig.add_trace(go.Scatter3d(x=flip_pca[:, 0], y=flip_pca[:, 1], z=flip_pca[:, 2],
                                mode='markers', marker=dict(color='orange'), name='intrahemispheric flipped references'))

fig.add_trace(go.Scatter3d(x=centroids[:, 0], y=centroids[:, 1], z=centroids[:, 2],
                                mode='markers+text', marker=dict(symbol='x', size=6),
                            text=[str(i) for i in range(num_clusters)], name='Centroids', showlegend=False))
for i in range(len(rmd_idx)):
    fig.add_trace(go.Scatter3d(
        x=[reduced_data[rmd_idx[i], 0], flip_pca[i, 0]],
        y=[reduced_data[rmd_idx[i], 1], flip_pca[i, 1]],
        z=[reduced_data[rmd_idx[i], 2], flip_pca[i, 2]],
        mode='lines',
        line=dict(color='blue'),
        showlegend=False
    ))


fig.update_layout(title=f"PCA-reduced connectivity matrices afetr K-means clustering  \n"
    "Centroids are marked with cross\n"
    f"Arrows indicate temporal order", 
    height=1000, width=1000)
fig.show()



In [ ]:
plt.imshow(correlation_matrices[0])

In [ ]:
plt.imshow(fn.square_flip(correlation_matrices[0]))

In [ ]:
components = pca.components_
for comp in components : 
    plt.figure()
    plt.imshow(comp.reshape(76,76),cmap='bwr')
    plt.colorbar();plt.clim([-0.05,0.05])

## distance to cluster + z-axis coordinates

In [ ]:
from scipy.signal import savgol_filter
distances = np.zeros((num_clusters,len(correlation_matrices)))
time = np.linspace(1,2400000/1000,len(correlation_matrices))
for j in range((num_clusters)):
    for i in range(len(correlation_matrices)):
        distances[j,i] = np.sum(np.abs(correlation_matrices[i]-most_repeating_states[j]))

sums=np.sum(distances,axis=0)

for i in range(len(correlation_matrices)):
    distances[:,i] = 1-distances[:,i]/sums[i]

colors=['red','blue','green','orange','black','purple']

plt.figure(figsize=[15,10])
plt.plot(time,savgol_filter(reduced_data[:-num_clusters, 2]/np.max(reduced_data[:, 2]), 40, 4),label='Smoothed relative PCA z-coordinate')
for p in range((num_clusters)):
    plt.plot(time,distances[p,:],label=str(p),c=colors[p])
plt.title('1-Relative distance to each cluster over time')
plt.xlabel('Time [s]')
plt.ylabel('1 - Relative distance')
plt.legend()

plt.show()

plt.figure(figsize=[15,10])
plt.scatter(time, cluster_labels, c = [colors[clust] for clust in cluster_labels], label=cluster_labels)
plt.yticks(np.arange(0,num_clusters,1))
plt.xlabel('time [s]')
plt.ylabel('State')
plt.title('Clustered state according to time')
plt.show()

## 5 min sim

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/20minsim_dt_0.5_G_{7}_sigma_1e-07.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 600000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth, window_size=10000,overlap=5000,tmax=t2/1000,animated_visu=False)

# Manual cluster creation

In [ ]:
for mat in most_repeating_states : 
    plt.figure()
    plt.imshow(mat)

In [ ]:
# 3D PCA
num_clusters=3
manual_clusters = [most_repeating_states[0],most_repeating_states[1],pca.inverse_transform([10,-5.5,0]).reshape(76,76),pca.inverse_transform([10,7,0]).reshape(76,76)]
ext_data=np.concatenate((correlation_matrices,manual_clusters))
data=np.array(ext_data).reshape(len(correlation_matrices)+4,5776)
pca=PCA(n_components=3)
reduced_data = pca.fit_transform(data)
manual_clusters = [most_repeating_states[0],most_repeating_states[1],pca.inverse_transform([10,-5.5,0]).reshape(76,76),pca.inverse_transform([10,7,0]).reshape(76,76)]

centroids = reduced_data[-4:,:]
labels=['right','left','top','bottom']

fig = go.Figure(data=[go.Scatter3d(x=reduced_data[:-4, 0], y=reduced_data[:-4, 1], z=reduced_data[:-4, 2],
                                mode='markers', marker=dict(color='black', size=3))])


fig.add_trace(go.Scatter3d(x=centroids[:, 0], y=centroids[:, 1], z=centroids[:, 2],
                                mode='markers+text', marker=dict(symbol='x', size=7),
                            text=[labels[i] for i in range(len(labels))], name='Centroids', showlegend=False))




fig.update_layout(title=f"PCA-reduced connectivity matrices afetr K-means clustering  \n"
    "Manually added Centroids are marked with cross\n", 
    height=1000, width=1000)
fig.show()



In [ ]:
plt.imshow(pca.inverse_transform([10,-5.5,0]).reshape(76,76))

In [ ]:
from scipy.signal import savgol_filter
distances = np.zeros((4,len(correlation_matrices)))
time = np.linspace(1,2400000/1000,len(correlation_matrices))
for j in range((4)):
    for i in range(len(correlation_matrices)):
        distances[j,i] = np.sum(np.abs(correlation_matrices[i]-manual_clusters[j]))

sums=np.sum(distances,axis=0)
labels=['right','left','top','bottom']

for i in range(len(correlation_matrices)):
    distances[:,i] = 1-distances[:,i]/sums[i]
colors=['red','blue','green','orange','black','purple']

plt.figure(figsize=[15,10])
plt.plot(time,savgol_filter(reduced_data[:-4, 2]/np.max(reduced_data[:-4, 2]), 50, 3),label='Smoothed relative PCA z-coordinate')
for p in range((4)):
    plt.plot(time,distances[p,:],label=labels[p],c=colors[p])
plt.title('1-Relative distance to each cluster over time')
plt.xlabel('Time [s]')
plt.ylabel('1 - Relative distance')
plt.legend()
plt.show()

for p in range((4)):
    plt.figure()
    df=pd.DataFrame(columns=['distances','smoothed z-coord'])
    df['distances']=distances[p,:]
    df['z-coord']=reduced_data[:-4, 2]/np.max(reduced_data[:-4, 2])
    sns.lmplot(x='distances',y='z-coord', data=df,height=6, aspect=1.5)
    plt.text(0.4,0.6,f'R squared = {np.round(scipy.stats.pearsonr(distances[p,:], reduced_data[:-4, 2]/np.max(reduced_data[:-4, 2]))[0],4)}')
    plt.show()




'''plt.figure(figsize=[15,10])
plt.scatter(time, cluster_labels, c = [colors[clust] for clust in cluster_labels], label=cluster_labels)
plt.yticks(np.arange(0,4,1))
plt.xlabel('time [s]')
plt.ylabel('State')
plt.title('Clustered state according to time')
plt.show()'''

In [ ]:
from scipy.signal import savgol_filter
distances = np.zeros((4,len(correlation_matrices)))
time = np.linspace(1,2400000/1000,len(correlation_matrices))
for j in range((4)):
    for i in range(len(correlation_matrices)):
        distances[j,i] = np.sum(np.abs(correlation_matrices[i]-manual_clusters[j]))

sums=np.sum(distances,axis=0)
labels=['right','left','top','bottom']

for i in range(len(correlation_matrices)):
    distances[:,i] = 1-distances[:,i]/sums[i]
colors=['red','blue','green','orange','black','purple']

plt.figure(figsize=[15,10])
plt.plot(time,savgol_filter(reduced_data[:-4, 1]/np.max(reduced_data[:-4, 1]), 50, 3),label='Smoothed relative PCA z-coordinate')
for p in range((4)):
    plt.plot(time,distances[p,:],label=labels[p],c=colors[p])
plt.title('1-Relative distance to each cluster over time')
plt.xlabel('Time [s]')
plt.ylabel('1 - Relative distance')
plt.legend()
plt.show()

for p in range((4)):
    plt.figure()
    df=pd.DataFrame(columns=['distances','smoothed z-coord'])
    df['distances']=distances[p,:]
    df['z-coord']=reduced_data[:-4, 1]/np.max(reduced_data[:-4, 1])
    sns.lmplot(x='distances',y='z-coord', data=df,height=6, aspect=1.5)
    plt.text(0.4,0.6,f'R squared = {np.round(scipy.stats.pearsonr(distances[p,:], reduced_data[:-4, 1]/np.max(reduced_data[:-4, 1]))[0],4)}')
    plt.show()




'''plt.figure(figsize=[15,10])
plt.scatter(time, cluster_labels, c = [colors[clust] for clust in cluster_labels], label=cluster_labels)
plt.yticks(np.arange(0,4,1))
plt.xlabel('time [s]')
plt.ylabel('State')
plt.title('Clustered state according to time')
plt.show()'''

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
gc_res=[]
for p in range((4)):
    print(labels[p])
    df=pd.DataFrame(columns=['distances','z-coord'])
    df['distances']=distances[p,:]
    df['z-coord']=reduced_data[:-4, 1]/np.max(reduced_data[:-4, 1])
    gc_res.append(grangercausalitytests(df.values, 4,verbose=1))
    


In [ ]:
print(gc_res[0])

In [ ]:
for i in manual_clusters : 
    plt.figure()
    plt.imshow(i)

# Noise level effect investigation

In [ ]:
sigma = [1e-3,1e-5,1e-6] 
for s in sigma : 
    data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{s}.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 2400000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances = DFC_visu_pipeline(data_corr,meth, window_size=10000,overlap=5000,tmax=t2/1000)

In [ ]:
sigma = np.linspace(1e-6,1e-7,20)#[1e-5,1e-6,9e-7,5e-7,3e-7,2e-7,1.5e-7,1e-7] 
for s in sigma : 
    data = np.load(f'{path_save}/testsigmaminsim_dt_0.5_G_{7}_sigma_{s}.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 2400000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    fn.build_FCmat(data_corr,'Pearson',num_clusters=3,window_size=2000,overlap=1000,tmax=30)

## smaller noise

In [ ]:
sigma = np.linspace(1e-12,0,20) #sigma = np.linspace(1e-7,1e-12,20)#[1e-5,1e-6,9e-7,5e-7,3e-7,2e-7,1.5e-7,1e-7] 
for s in sigma : 
    data = np.load(f'{path_save}/testsigmaminsim_dt_0.5_G_{7}_sigma_{np.round(s,3)}.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 2400000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    fn.build_FCmat(data_corr,'Pearson',num_clusters=3,window_size=2000,overlap=1000,tmax=30)

In [ ]:
sigma = [1e-10,1e-12,0] 
for s in sigma : 
    data = np.load(f'{path_save}/testsigmaminsim_dt_0.5_G_{7}_sigma_{s}.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 2400000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances = DFC_visu_pipeline(data_corr,meth,window_size=2000,overlap=1000)

## z to intrahemispheric relation

In [ ]:
from scipy.signal import savgol_filter
sums=np.sum(distances,axis=0)
labels=['right','left','top','bottom']

for i in range(len(correlation_matrices)):
    distances[:,i] = 1-distances[:,i]/sums[i]
colors=['red','blue','green','orange','black','purple']

plt.figure(figsize=[15,10])
plt.plot(time,savgol_filter(reduced_data[:-3, 1]/np.max(reduced_data[:-3, 1]), 50, 3),label='Smoothed relative PCA z-coordinate')
for p in range((3)):
    plt.plot(time,distances[p,:],label=labels[p],c=colors[p])
plt.title('1-Relative distance to each cluster over time')
plt.xlabel('Time [s]')
plt.ylabel('1 - Relative distance')
plt.legend()
plt.show()

for p in range((3)):
    plt.figure()
    df=pd.DataFrame(columns=['distances','smoothed z-coord'])
    df['distances']=distances[p,:]
    df['z-coord']=reduced_data[:-3, 1]/np.max(reduced_data[:-3, 1])
    sns.lmplot(x='distances',y='z-coord', data=df,height=6, aspect=1.5)
    plt.text(0.4,0.6,f'R squared = {np.round(scipy.stats.pearsonr(distances[p,:], reduced_data[:-3, 1]/np.max(reduced_data[:-3, 1]))[0],4)}')
    plt.show()




'''plt.figure(figsize=[15,10])
plt.scatter(time, cluster_labels, c = [colors[clust] for clust in cluster_labels], label=cluster_labels)
plt.yticks(np.arange(0,4,1))
plt.xlabel('time [s]')
plt.ylabel('State')
plt.title('Clustered state according to time')
plt.show()'''

# Study on subjects connectivity : subject TVB2

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB2.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 300000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca2, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

In [ ]:
def pca_explained_variance(pca):
    return np.cumsum(pca.explained_variance_ratio_)

In [ ]:
print(pca_explained_variance(pca))

In [ ]:
pca.explained_variance_ratio_

# TVB10

In [ ]:

G=[7]
sigma=[1e-9]
#G=[1,1.5,2,3,4,5,6,7,8,9,10,12]
for s in range(len(sigma)):
    data = np.load(f'{path_save}/testsigmaminsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB10.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 300000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

## 40 minutes

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'

G=[5,7]
sigma = [1e-7,1e-9,1e-11]
#G=[1,1.5,2,3,4,5,6,7,8,9,10,12]
for g in range(len(G)):
    for s in range(len(sigma)):
        print(G[g],' ',sigma[s])
        data = np.load(f'{path_save}/40sim_dt_0.5_G_{G[g]}_sigma_{sigma[s]}_TVB10.npz')
        data_time=data['traw_time']
        delta_t = data_time[1]-data_time[0]
        sf = 1/(delta_t*1e-3)
        win = 1*sf

        t1 = 1000
        t2 = 2400000 #unit second
        time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
        idc1 = time_corr_id[0][0]
        idc2= time_corr_id[0][-1]
        time_corr=data_time[idc1:idc2]
        data_corr=data['traw_y1y2'][idc1:idc2]
        print(np.array(data_corr).shape)

        methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
        for meth in methods : 
            closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=10000,overlap=5000,tmax=t2/1000, intrahemispheric=False)

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40sim_dt_0.5_G_{7}_sigma_{1e-9}_TVB10.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 2400000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca10, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=10000,overlap=5000,tmax=t2/1000, intrahemispheric=True, square=[1,1])

In [ ]:
animated_plot_cloud(correlation_matrices, reduced_data, cluster_labels)

In [ ]:
plot_PCs(pca)

### Intrahemispheric

In [ ]:
data = np.load(f'{path_save}/40sim_dt_0.5_G_{7}_sigma_{1e-09}_TVB10.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 2400000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances = DFC_visu_pipeline(data_corr,meth,intrahemispheric=False,square=[0,1], window_size=100, overlap=50)

### PCA ref transform

In [ ]:
# 3D PCA
num_clusters=3
np.random.seed=42
ext_data=np.concatenate((correlation_matrices,most_repeating_states))
data=np.array(ext_data).reshape(len(correlation_matrices)+num_clusters,len(correlation_matrices[0])**2)
#flipped_mtc=[inter_flip(mtx) for mtx in np.array(correlation_matrices)[rmd_idx]]
pre_reduced= PCA(n_components=76**2).fit_transform(data)
reduced_data = pca_ref.transform(pre_reduced)
centroids = reduced_data[-num_clusters:]

fig = go.Figure(data=[go.Scatter3d(x=reduced_data[:-num_clusters, 0], y=reduced_data[:-num_clusters, 1], z=reduced_data[:-num_clusters, 2],
                                    mode='markers', marker=dict(color=cluster_labels))])

centroids = reduced_data[-num_clusters:]

fig.add_trace(go.Scatter3d(x=centroids[:, 0], y=centroids[:, 1], z=centroids[:, 2],
                                mode='markers+text', marker=dict(symbol='x', size=6),
                            text=[str(i) for i in range(num_clusters)], name='Centroids', showlegend=False))

fig.add_trace(go.Scatter3d(x=reduced_data[:-num_clusters, 0],y=reduced_data[:-num_clusters, 1],z=reduced_data[:-num_clusters, 2],mode='lines',line=dict(color='black', width=2)))


fig.update_layout(title=f"PCA-reduced connectivity matrices afetr K-means clustering  \n"
    "Centroids are marked with cross\n"
    f"Arrows indicate temporal order", 
    height=1000, width=1000)
fig.show()



# TVB5

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB5.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 300000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

In [ ]:
plot_PCs(pca)

## 40 min

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB5.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 2400000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca5, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=10000,overlap=5000,tmax=t2/1000, intrahemispheric=False)

In [ ]:
animated_plot_cloud(correlation_matrices, reduced_data, cluster_labels)

In [ ]:
plot_PCs(pca)

## Intrahemispheric

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB5.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 2400000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca5, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=10000,overlap=5000,tmax=t2/1000, intrahemispheric=True, square=[1,1])

In [ ]:
animated_plot_cloud(correlation_matrices, reduced_data, cluster_labels)

# TVB7

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB7.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 300000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

In [ ]:
animated_plot_cloud(correlation_matrices, reduced_data, cluster_labels)

In [ ]:
plot_PCs(pca)

## Try normalisation 

In [ ]:
from sklearn import preprocessing

path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB7.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 300000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
data_corr = preprocessing.normalize(data_corr, axis=0)
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca7, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

## Try Granger

In [ ]:
import numpy as np
from statsmodels.tsa.api import VAR

# Initialize the directed functional connectivity matrix
connectivity_matrix = np.zeros((84,84))
data = data_corr
windowed_data = data[1000:1000 + int(2000), :]

# Fit a VAR model to the data
model = VAR(windowed_data)

# Compute the lag order using an information criterion (e.g., AIC or BIC)
lag_order = 4

# Estimate the coefficients of the VAR model
model_fit = model.fit(lag_order)

# Compute the Granger causality matrix
granger_matrix = model_fit.test_causality(caused=range(84), causing=range(84), kind='f')

# Update the directed functional connectivity matrix with the Granger causality values
connectivity_matrix = granger_matrix.reshape(84, 84)

# Print the directed functional connectivity matrix
print(connectivity_matrix)


In [ ]:
len(windowed_data.T)

In [ ]:
granger_matrix.summary()

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

tst=np.zeros(84*84)

for i in range(84):
    for j in range(84):
        df=pd.DataFrame(columns=['1','2'])
        df['1']=windowed_data[:,i]
        df['2']=windowed_data[:,j]
        x=grangercausalitytests(df.values, [2],verbose=1)
        tst[i*84+j]=x[2][0]['params_ftest'][1]
    

In [ ]:
plt.imshow(tst.reshape(84,84))
plt.colorbar()

In [ ]:
print(x)

# TVB8

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB8.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 300000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

In [ ]:
print(pca_explained_variance(pca))


# TVB9

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB9.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 300000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca2, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

In [ ]:
print(pca_explained_variance(pca))


# TVB11

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB11.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 2400000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca11, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=10000,overlap=5000,tmax=t2/1000, intrahemispheric=False)

In [ ]:
print(pca_explained_variance(pca))


# TVB12

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40minsim_dt_0.5_G_{7}_sigma_{1e-9}_TVB12.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 2400000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca12, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=10000,overlap=5000,tmax=t2/1000, intrahemispheric=False)

In [ ]:
print(pca_explained_variance(pca))


# PCs comparison

In [ ]:
pcas=[pca2,pca5,pca7,pca8,pca9,pca10,pca11,pca12]
str_pcas=['pca2','pca5','pca7','pca8','pca9','pca10','pca11','pca12']
comps=[]
for p,pca in enumerate(pcas) : 
    comps.append(plot_PCs(pca))

In [ ]:
sp_comps = []
for c in comps : 
    for cc in c : 
        sp_comps.append(cc)
        
 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-0.05, 0.05))
sp_comps = scaler.fit_transform(sp_comps)
for i, c in enumerate(sp_comps): 
    sp_comps = scaler.fit_transform(sp_comps)
    for j, com in enumerate(sp_comps): 
        if i != j: 
            diff = c - com
            plt.figure()
            plt.imshow(diff.reshape(84, 84),cmap='bwr')
            plt.colorbar();plt.clim([-0.05,0.05])
            plt.title(f'PC{np.floor(i/3)+1}, {str_pcas[i%3]} - PC{np.floor(j/3)+1}, {str_pcas[j%3]}')


# Angle evolution

In [ ]:
import numpy as np

def calculate_angle(point1, point2):
    vector1 = np.array(point1)
    vector2 = np.array(point2)
    
    dot_product = np.dot(vector1, vector2)
    magnitude_product = np.linalg.norm(vector1) * np.linalg.norm(vector2)
    
    angle = np.arccos(dot_product / magnitude_product)
    
    return np.degrees(angle)



In [ ]:
plt.plot([calculate_angle(reduced_data[i,:],reduced_data[i+1,:]) for i in range(len(reduced_data)-1)])

In [ ]:

def calculate_phase(point1, point2):
    phase1 = np.angle(point1,deg=True)
    phase2 = np.angle(point2,deg=True)
    
    phase_diff = phase2 - phase1
    
    return phase_diff


angles = [calculate_angle(reduced_data[i,:],reduced_data[i+1,:]) for i in range(len(reduced_data)-1)]
phases = [calculate_phase(reduced_data[i,:],reduced_data[i+1,:]) for i in range(len(reduced_data)-1)]

plt.scatter(angles, phases, c=phases, cmap='coolwarm')
plt.colorbar(label='Phase Difference')
plt.xlabel('Angle')
plt.ylabel('Phase Difference')
plt.title('Angle vs Phase Difference')
plt.show()


In [ ]:
angles = [calculate_angle(reduced_data[i,:],reduced_data[i+1,:]) for i in range(len(reduced_data)-1)]


fig = go.Figure(data=[go.Scatter3d(x=reduced_data[:-num_clusters, 0], y=reduced_data[:-num_clusters, 1], z=reduced_data[:-num_clusters, 2],
                                mode='markers', marker=dict(color='black',size=2))])

centroids = reduced_data[-num_clusters:]

fig.add_trace(go.Scatter3d(x=centroids[:, 0], y=centroids[:, 1], z=centroids[:, 2],
                                mode='markers+text', marker=dict(symbol='x', size=6),
                            text=[str(i) for i in range(num_clusters)], name='Centroids', showlegend=False))

fig.add_trace(go.Scatter3d(x=reduced_data[:-num_clusters, 0],y=reduced_data[:-num_clusters, 1],z=reduced_data[:-num_clusters, 2],mode='lines',line=dict(color=angles*100, width=2)))


fig.update_layout(title=f"PCA-reduced connectivity matrices afetr K-means clustering  \n"
    "Centroids are marked with cross\n"
    f"points are linked according to temporal order", 
    height=1000, width=1000)
fig.show()

In [ ]:
angles

# New plan projection Esra

## TVB10

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/40sim_dt_0.5_G_{7}_sigma_{1e-9}_TVB10.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 300000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca10, reduced_data = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False, square=[1,1])

In [ ]:
components = pca10.components_

In [ ]:
new_PC1 = components[0]
new_PC2 = components[1]-components[2]
new_PC3 = components[1]+components[2]
new_PCs=np.array([new_PC1,new_PC2,new_PC3])

for PC in new_PCs : 
    plt.figure()
    plt.imshow(PC.reshape(84,84),cmap='bwr')
    plt.colorbar();plt.clim([-0.05,0.05])

new_PCs=pca10.transform(new_PCs)
new_PC1=new_PCs[0]
new_PC2=new_PCs[1]
new_PC3=new_PCs[2]

In [ ]:
def define_plane(point1, point2):
    # Calculate the normal vector of the plane
    normal = np.cross(point1, point2)
    return normal

# Function to project points onto the plane
def project_onto_plane(plane_normal, plane_point, other_points):
    # Ensure the plane normal is a unit vector
    plane_normal = plane_normal / np.linalg.norm(plane_normal)
    
    # Calculate the equation of the plane (ax + by + cz = d)
    d = np.dot(plane_normal, plane_point)
    
    # Calculate the projection of other points onto the plane
    projections = []
    for point in other_points:
        t = d - np.dot(plane_normal, point)
        projected_point = point + t * plane_normal
        projections.append(projected_point)
    
    return projections

In [ ]:
plan1=define_plane(new_PC1,new_PC2)
plan2=define_plane(new_PC1,new_PC3)
plan3=define_plane(new_PC2,new_PC3)

proj1=np.array(project_onto_plane(plan1, new_PC1, reduced_data))
proj2=np.array(project_onto_plane(plan2, new_PC1, reduced_data))
proj3=np.array(project_onto_plane(plan3, new_PC2, reduced_data))

### PLAN 1

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=proj1[:-num_clusters, 0], y=proj1[:-num_clusters, 1], z=proj1[:-num_clusters, 2],
                                    mode='markers', marker=dict(color=cluster_labels))])

centroids = proj1[-num_clusters:]

fig.add_trace(go.Scatter3d(x=centroids[:, 0], y=centroids[:, 1], z=centroids[:, 2],
                                mode='markers+text', marker=dict(symbol='x', size=6),
                            text=[str(i) for i in range(num_clusters)], name='Centroids', showlegend=False))

fig.add_trace(go.Scatter3d(x=proj1[:-num_clusters, 0],y=proj1[:-num_clusters, 1],z=proj1[:-num_clusters, 2],mode='lines',line=dict(color='black', width=2)))


fig.update_layout(title=f"Projected PCA; axis1=PC1/ axis2=PC2-PC3", 
    height=1000, width=1000)
fig.show()

In [ ]:
def project_3d_to_2d(points_3d, plane_normal):
    # Ensure the plane normal is a unit vector
    plane_normal = plane_normal / np.linalg.norm(plane_normal)
    
    # Calculate the projection of points onto the plane
    # The projection formula: P = V - dot(V, N) * N, where V is the point and N is the normal vector
    projected_points = points_3d - np.dot(points_3d, plane_normal)[:, np.newaxis] * plane_normal
    
    return projected_points

proj1_2d=pd.DataFrame(project_3d_to_2d(proj1,np.cross(proj1[0],proj1[1])))

plt.scatter(proj1_2d[0][:-num_clusters],proj1_2d[1][:-num_clusters],c=cluster_labels)

In [ ]:
colorscale = ['#7A4579', '#D56073', 'rgb(236,158,105)', (1, 1, 0.2), (0.98,0.98,0.98)]

fig = ff.create_2d_density(
    proj1_2d[0][:-num_clusters],proj1_2d[1][:-num_clusters], colorscale=colorscale,
    hist_color='rgb(255, 237, 222)', point_size=3
)
fig.update_layout(title=f"Desity plot of projected points", 
    height=1000, width=1000)

# Add crosses marking the projections of the centroids
centroids_proj = project_3d_to_2d(centroids, np.cross(proj1[0], proj1[1]))
fig.add_trace(go.Scatter(x=centroids_proj[:, 0], y=centroids_proj[:, 1],
                         mode='markers+text', marker=dict(symbol='x', size=20,color='white'),text=[str(i) for i in range(num_clusters)],
                         
                         name='Centroids'))



fig.update_layout(title=f"Desity plot of projected points", 
    height=1000, width=1000)
fig.show()

In [ ]:
manual_center=[-0.5,0.5]
xs=proj1_2d[:-num_clusters][0]
ys=proj1_2d[:-num_clusters][1]

dx=manual_center[0]-xs
dy=manual_center[1]-ys

radial_distance=np.sqrt(dx**2+dy**2)
angles=np.arctan2(dy,dx)
unwraped=np.unwrap(angles)
indeg=np.degrees(angles)

In [ ]:
fig=go.Figure(data=[go.Scatter(x=list(range(len(unwraped))),y=unwraped,mode='markers',marker=dict(color=radial_distance,size=5,showscale=True))])
fig.update_layout(coloraxis=dict(colorbar=dict(title='Radial Distance')))
fig.update_layout(title=f"Angle evolution, Subj10, Healthy", 
    height=1000, width=1750)
fig.show()

### PLAN 2

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=proj2[:-num_clusters, 0], y=proj2[:-num_clusters, 1], z=proj2[:-num_clusters, 2],
                                    mode='markers', marker=dict(color=cluster_labels))])

centroids = proj2[-num_clusters:]

fig.add_trace(go.Scatter3d(x=centroids[:, 0], y=centroids[:, 1], z=centroids[:, 2],
                                mode='markers+text', marker=dict(symbol='x', size=6),
                            text=[str(i) for i in range(num_clusters)], name='Centroids', showlegend=False))

fig.add_trace(go.Scatter3d(x=proj2[:-num_clusters, 0],y=proj2[:-num_clusters, 1],z=proj2[:-num_clusters, 2],mode='lines',line=dict(color='black', width=2)))


fig.update_layout(title=f"Projected PCA-reduced connectivity matrices afetr K-means clustering  \n"
    "Centroids are marked with cross\n"
    f"points are linked according to temporal order", 
    height=1000, width=1000)
fig.show()


In [ ]:
proj2_2d=pd.DataFrame(project_3d_to_2d(proj2,np.cross(proj2[0],proj2[4])))

plt.scatter(proj2_2d[0][:-num_clusters],proj2_2d[1][:-num_clusters],c=cluster_labels)

### PLAN 3

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=proj3[:-num_clusters, 0], y=proj3[:-num_clusters, 1], z=proj3[:-num_clusters, 2],
                                    mode='markers', marker=dict(color=cluster_labels))])

centroids = proj3[-num_clusters:]

fig.add_trace(go.Scatter3d(x=centroids[:, 0], y=centroids[:, 1], z=centroids[:, 2],
                                mode='markers+text', marker=dict(symbol='x', size=6),
                            text=[str(i) for i in range(num_clusters)], name='Centroids', showlegend=False))

fig.add_trace(go.Scatter3d(x=proj3[:-num_clusters, 0],y=proj3[:-num_clusters, 1],z=proj3[:-num_clusters, 2],mode='lines',line=dict(color='black', width=2)))


fig.update_layout(title=f"Projected PCA-reduced connectivity matrices afetr K-means clustering  \n"
    "Centroids are marked with cross\n"
    f"points are linked according to temporal order", 
    height=1000, width=1000)
fig.show()


In [ ]:
plt.scatter(proj3[:-num_clusters, 2],proj3[:-num_clusters, 0])

In [ ]:
colorscale = ['#7A4579', '#D56073', 'rgb(236,158,105)', (1, 1, 0.2), (0.98,0.98,0.98)]

fig = ff.create_2d_density(
    proj3[:-num_clusters, 1], proj3[:-num_clusters, 2], colorscale=colorscale,
    hist_color='rgb(255, 237, 222)', point_size=3
)
plt.scatter()
fig.update_layout(title=f"Projected PCA-reduced connectivity matrices afetr K-means clustering  \n"
    "Centroids are marked with cross\n"
    f"points are linked according to temporal order", 
    height=1000, width=1000)
fig.show()

In [ ]:
con1=proj3[:, 1]/proj3[:, 0]
con2=proj3[:, 2]/proj3[:, 0]
con=np.array([con1,con2])

In [ ]:
plt.scatter(con[0,:-num_clusters],con[1,:-num_clusters],c=cluster_labels)

## Template

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/20minsim_dt_0.5_G_{7}_sigma_1e-07.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 2400000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth, window_size=10000,overlap=5000,tmax=t2/1000)

In [ ]:
ext_data=np.concatenate((correlation_matrices,most_repeating_states))
data=np.array(ext_data).reshape(len(correlation_matrices)+num_clusters,len(correlation_matrices[0])**2)
reduced_data_2d = PCA(n_components=2).fit_transform(data)
manual_center=[6,0]

colorscale = ['#7A4579', '#D56073', 'rgb(236,158,105)', (1, 1, 0.2), (0.98,0.98,0.98)]

fig = ff.create_2d_density(
    reduced_data_2d[:-num_clusters, 0], reduced_data_2d[:-num_clusters, 1], colorscale=colorscale,
    hist_color='rgb(255, 237, 222)', point_size=3
)
fig.add_trace(go.Scatter(x=[manual_center[0]],y=[manual_center[1]],mode='markers',marker=dict(symbol='x',color='black',size=10)))

fig.update_layout(title=f"Projected PCA-reduced connectivity matrices afetr K-means clustering  \n"
    "Centroids are marked with cross\n"
    f"points are linked according to temporal order", 
    height=1000, width=1000)
fig.show()

### Angle and radial distance evolution

In [ ]:
manual_center=[6,0]
xs=reduced_data_2d[:-num_clusters, 0]
ys=reduced_data_2d[:-num_clusters,1]

dx=manual_center[0]-xs
dy=manual_center[1]-ys

radial_distance=np.sqrt(dx**2+dy**2)
angles=np.arctan2(dy,dx)
unwraped=np.unwrap(angles)
indeg=np.degrees(angles)

In [ ]:
plt.plot(indeg)
plt.xlabel('time')
plt.ylabel('angle')
plt.title('Angle of each point with respect to the ellipse center')
plt.figure()
plt.plot(radial_distance)
plt.xlabel('time')
plt.ylabel('radial distance')
plt.title('Radial distance of each point accross time')

In [ ]:
plt.plot(unwraped)
plt.ylabel('unwraped angle')

In [ ]:
fig=go.Figure(data=[go.Scatter(x=list(range(len(unwraped))),y=unwraped,mode='markers',marker=dict(color=radial_distance,size=5,showscale=True))])
fig.update_layout(coloraxis=dict(colorbar=dict(title='Radial Distance')))
fig.update_layout(title=f"Angle evolution", 
    height=1000, width=1750)
fig.show()

In [ ]:
plt.plot(radial_distance,indeg)
plt.title('redial distance vs angle')

### Corr corr Matrix + dendrogram

In [ ]:
corr_corr_mat=np.corrcoef(np.array(correlation_matrices).reshape(len(correlation_matrices),len(correlation_matrices[0])**2))

In [ ]:
plt.imshow(corr_corr_mat)
plt.colorbar()

In [ ]:
linkage_data = linkage(corr_corr_mat, method='ward', metric='euclidean')


In [ ]:
plt.figure(figsize=(25, 10))
k=8
dend=dendrogram(linkage_data, orientation='top', color_threshold=8)  
clusters=scipy.cluster.hierarchy.fcluster(linkage_data, 8, criterion='maxclust')

plt.show()


In [ ]:
plt.figure(figsize=(25, 10))
dend=dendrogram(linkage_data, orientation='top', truncate_mode='lastp', p=8, show_leaf_counts=True, show_contracted=True, no_labels=True, color_threshold=100)  
plt.show()

In [ ]:
fig=ff.create_dendrogram(corr_corr_mat, color_threshold=4.5)
fig.update_layout(width=800, height=500)
fig.show()

In [ ]:
testi=scipy.cluster.hierarchy.fcluster(linkage_data, 8, criterion='maxclust')

In [ ]:
clust1=[]
clust2=[]
clust3=[]
clust4=[]
clust5=[]
clust6=[]
clust7=[]
clust8=[]

clusts=[ [] for _ in range(8) ]

for i in range(len(correlation_matrices)):
    clusts[clusters[i]-1].append(correlation_matrices[i])

for j in range(len(clusts)):
    plt.figure()
    plt.imshow(np.mean(clusts[j],axis=0))

In [ ]:
reorg_matrices = [clusts[0]+clusts[1]+clusts[2]+clusts[3]+clusts[4]+clusts[5]+clusts[6]+clusts[7]] 

In [ ]:
clust1=np.zeros(len(testi[testi==1]))
clust2=np.zeros(len(testi[testi==2]))
clust3=np.zeros(len(testi[testi==3]))
clust4=np.zeros(len(testi[testi==4]))
clust5=np.zeros(len(testi[testi==5]))
clust6=np.zeros(len(testi[testi==6]))
clust7=np.zeros(len(testi[testi==7]))
clust8=np.zeros(len(testi[testi==8]))

for i in range(l)


In [ ]:
import plotly.graph_objects as go
import plotly.figure_factory as ff

import numpy as np
from scipy.spatial.distance import pdist, squareform


# get data
data = np.genfromtxt("http://files.figshare.com/2133304/ExpRawData_E_TABM_84_A_AFFY_44.tab",
                     names=True,usecols=tuple(range(1,30)),dtype=float, delimiter="\t")
data_array = data.view((float, len(data.dtype.names)))
data_array = corr_corr_mat


# Initialize figure by creating upper dendrogram
fig = ff.create_dendrogram(data_array, orientation='bottom')
for i in range(len(fig['data'])):
    fig['data'][i]['yaxis'] = 'y2'

# Create Side Dendrogram
dendro_side = ff.create_dendrogram(data_array, orientation='right')
for i in range(len(dendro_side['data'])):
    dendro_side['data'][i]['xaxis'] = 'x2'

# Add Side Dendrogram Data to Figure
for data in dendro_side['data']:
    fig.add_trace(data)

# Create Heatmap
dendro_leaves = dendro_side['layout']['yaxis']['ticktext']
dendro_leaves = list(map(int, dendro_leaves))
data_dist = pdist(data_array)
heat_data = squareform(data_dist)
heat_data = heat_data[dendro_leaves,:]
heat_data = heat_data[:,dendro_leaves]

heatmap = [
    go.Heatmap(
        x = dendro_leaves,
        y = dendro_leaves,
        z = heat_data,
        colorscale = 'Blues'
    )
]

heatmap[0]['x'] = fig['layout']['xaxis']['tickvals']
heatmap[0]['y'] = dendro_side['layout']['yaxis']['tickvals']

# Add Heatmap Data to Figure
for data in heatmap:
    fig.add_trace(data)

# Edit Layout
fig.update_layout({'width':2000, 'height':2000,
                         'showlegend':False, 'hovermode': 'closest',
                         })
# Edit xaxis
fig.update_layout(xaxis={'domain': [.15, 1],
                                  'mirror': False,
                                  'showgrid': False,
                                  'showline': False,
                                  'zeroline': False,
                                  'ticks':""})
# Edit xaxis2
fig.update_layout(xaxis2={'domain': [0, .15],
                                   'mirror': False,
                                   'showgrid': False,
                                   'showline': False,
                                   'zeroline': False,
                                   'showticklabels': False,
                                   'ticks':""})

# Edit yaxis
fig.update_layout(yaxis={'domain': [0, .85],
                                  'mirror': False,
                                  'showgrid': False,
                                  'showline': False,
                                  'zeroline': False,
                                  'showticklabels': False,
                                  'ticks': ""
                        })
# Edit yaxis2
fig.update_layout(yaxis2={'domain':[.825, .975],
                                   'mirror': False,
                                   'showgrid': False,
                                   'showline': False,
                                   'zeroline': False,
                                   'showticklabels': False,
                                   'ticks':""})

# Plot!
fig.show()

# Timesteps investigation

## Timestep 0.5

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
data = np.load(f'{path_save}/20minsim_dt_0.5_G_{7}_sigma_1e-07.npz')
data_time=data['traw_time']
delta_t = data_time[1]-data_time[0]
sf = 1/(delta_t*1e-3)
win = 1*sf

t1 = 1000
t2 = 600000 #unit second
time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
idc1 = time_corr_id[0][0]
idc2= time_corr_id[0][-1]
time_corr=data_time[idc1:idc2]
data_corr=data['traw_y1y2'][idc1:idc2]
print(np.array(data_corr).shape)

methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
for meth in methods : 
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth, window_size=10000,overlap=5000,tmax=t2/1000)

## Timestep 0.3

In [ ]:
seeds = [40,50,60]
for s in seeds :
    path_conn = 'D:/Fariba/SC'
    path_save = f'D:/Fariba/Timestep_investigation'
    data = np.load(f'{path_save}/5minsim_dt_0.3_G_{7}_sigma_1e-07_seed{s}.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 600000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth, window_size=int(10000*0.5/0.3),overlap=int(5000*0.5/0.3),tmax=t2/1000,animated_visu=False)

## Timestep 0.2

In [ ]:
seeds = [40,50,60]
for s in seeds :
    path_conn = 'D:/Fariba/SC'
    path_save = f'D:/Fariba/Timestep_investigation'
    data = np.load(f'{path_save}/5minsim_dt_0.2_G_{7}_sigma_1e-07_seed{s}.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 600000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth, window_size=int(10000*0.5/0.2),overlap=int(5000*0.5/0.2),tmax=t2/1000,animated_visu=False)

## Timestep 0.1

In [ ]:
seeds = [40,50,60]
for s in seeds :
    path_conn = 'D:/Fariba/SC'
    path_save = f'D:/Fariba/Timestep_investigation'
    data = np.load(f'{path_save}/5minsim_dt_0.1_G_{7}_sigma_1e-07_seed{s}.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 600000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth, window_size=int(10000*0.5/0.1),overlap=int(5000*0.5/0.1),tmax=t2/1000,animated_visu=False)

## Timestep 0.05

In [ ]:
seeds = [40,50,60]
for s in seeds :
    path_conn = 'D:/Fariba/SC'
    path_save = f'D:/Fariba/Timestep_investigation'
    data = np.load(f'{path_save}/5minsim_dt_0.05_G_{7}_sigma_1e-07_seed{s}.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 600000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(data_corr,meth, window_size=int(10000*0.5/0.05),overlap=int(5000*0.5/0.05),tmax=t2/1000,animated_visu=False,ts=0.05)